In [20]:
# 0) Point to your CNN‑DAGMM folder *before* any imports
import sys, importlib
from pathlib import Path

cnn_dir = Path.cwd().parent / "cnn_dagmm"
sys.path.insert(0, str(cnn_dir))

# 1) Import & force‑reload to clear any old cache
import compression_network, model
importlib.reload(compression_network)
importlib.reload(model)

# 2) Verify you’re using the right files
print("CompressionNetwork from:", compression_network.__file__)
print("DAGMM   from:",           model.__file__)


CompressionNetwork from: /Users/aryan/Desktop/Academics /Semester 4/Data science/Project/gmm_anomalydetection/cnn_dagmm/compression_network.py
DAGMM   from: /Users/aryan/Desktop/Academics /Semester 4/Data science/Project/gmm_anomalydetection/cnn_dagmm/model.py


In [21]:
# %% [markdown]
# # DAGMM Anomaly Detection with Sampled Anomalies in Train

# %% [code]
# 1) Make your DAGMM code importable
import sys, os
from pathlib import Path
sys.path.insert(0, str(Path.cwd().parent / "cnn_dagmm"))

# %% [code]
# 2) Imports
import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import transforms, datasets
from collections import Counter

from model import DAGMM  # your revised model.py

# %% [code]
# 3) Data transforms & loaders (images → flattened vectors)
transform = transforms.Compose([
    transforms.Resize((64,64)),
    transforms.ToTensor(),
])

train_ds = datasets.ImageFolder("dataset/train", transform=transform, allow_empty=True)
test_ds  = datasets.ImageFolder("dataset/test",  transform=transform)

train_loader = DataLoader(train_ds, batch_size=1, shuffle=True,  drop_last=True)
test_loader  = DataLoader(test_ds,  batch_size=1, shuffle=False, drop_last=False)

# Print ground‑truth counts
print("Train class counts:", {train_ds.classes[k]: v for k,v in Counter(train_ds.targets).items()})
print("Test  class counts:", {test_ds.classes[k]: v for k,v in Counter(test_ds.targets).items()})

Train class counts: {'normal': 100}
Test  class counts: {'anomalous': 20, 'normal': 100}


In [22]:
# for imgs, labels in train_loader:
#     labels = labels.numpy()
#     print("Test batch labels:", labels)

In [23]:


# %% [code]
# 4) Model + optimizer
device = torch.device("mps" if torch.cuda.is_available() else "cpu")
n_features = 3 * 64 * 64

model = DAGMM(
    input_dim        = 3 * 64 * 64,     # still required by signature but not forwarded to CompressionNetwork
    latent_dim       = 90,
    n_gmm_components = 6,
    comp_kwargs      = {'latent_dim': 90},  # now cleanly matches CompressionNetwork
    est_kwargs       = {'hidden_dims': [128], 'activation': torch.nn.Tanh, 'dropout': 0.3},
    device           = device
).to(device)


optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

# %% [code]
# 5) Training loop
n_epochs = 50
for epoch in range(1, n_epochs+1):
    model.train()
    running_loss = 0.0
    for imgs, _ in train_loader:
        x = imgs.to(device) 
        out  = model(x)
        loss = model.loss_function(x, out)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * x.size(0)
    avg_loss = running_loss / len(train_loader.dataset)
    print(f"Epoch {epoch}/{n_epochs} — avg train loss: {avg_loss:.4f}")

# %% [code]
# 6) Scoring test set & thresholding
model.eval()
energies = []
with torch.no_grad():
    for imgs, _ in test_loader:
        x = imgs.to(device)
        energies.append(model(x)['energy'].cpu())
energies = torch.cat(energies)

# 95th‐percentile threshold
thr = energies.quantile(0.70)
mask = energies > thr
print(f"Detected anomalies in test set: {mask.sum().item()} / {len(energies)}")

# %% [code]
# 7) (Optional) Visualize
import matplotlib.pyplot as plt
plt.hist(energies.numpy(), bins=50, alpha=0.7)
plt.axvline(thr, color='r', linestyle='--', label='66% threshold')
plt.legend(); plt.title("Test Energy Distribution"); plt.show()


ValueError: Expected more than 1 value per channel when training, got input size torch.Size([1, 128])

In [ ]:
# %% [code]
# 6) Scoring test set & thresholding
model.eval()
energies = []
with torch.no_grad():
    for imgs, _ in test_loader:
        x = imgs.to(device)
        energies.append(model(x)['energy'].cpu())
energies = torch.cat(energies)

# 95th‐percentile threshold
thr = energies.quantile(0.90)
mask = energies > thr
print(energies)
print(mask)
print(f"Detected anomalies in test set: {mask.sum().item()} / {len(energies)}")

# # %% [code]
# # 7) (Optional) Visualize
# import matplotlib.pyplot as plt
# plt.hist(energies.numpy(), bins=50, alpha=0.7)
# plt.axvline(thr, color='r', linestyle='--', label='66% threshold')
# plt.legend(); plt.title("Test Energy Distribution"); plt.show()


tensor([ 709.9402,  709.9402, 1424.3522, 1418.5375, 1399.5629, 1370.5111,
         709.9402, 1363.6235,  709.9402, 1392.9539,  709.9402,  709.9402,
        1419.8672, 1431.4800,  709.9402,  709.9402,  709.9402,  709.9402,
         709.9402,  709.9402,  101.6475,  101.6475,  108.7146,  114.6900,
         108.7146,  101.7647,  108.7146,  123.6524,  101.6475,  101.7647,
         123.6524,  123.6524,  114.2931,  101.7647,  101.6475,  114.2931,
         108.7146,  101.6475,  101.7647,  108.7146,  108.7146,  123.6524,
         101.6475,  114.2931,  101.6475,  101.6475,  101.6475,  123.6524,
         108.7146,  108.7146,  114.6900,  114.6900,  123.6524,  123.6524,
         108.7146,  114.6900,  101.6475,  101.7647,  108.7146,  101.7647,
         114.2931,  101.6475,  101.6475,  101.7647,  101.7647,  114.6900,
         101.7647,  101.7647,  101.7647,  114.6900,  123.6524,  101.7647,
         108.7146,  123.6524,  101.7647,  123.6524,  108.7146,  114.6900,
         101.6475,  101.7647,  108.714

In [ ]:
import torch
from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score

# 1) Recompute energies and collect true labels
model.eval()
energies = []
y_true   = []
with torch.no_grad():
    for imgs, labels in test_loader:
        x = imgs.to(device)                       # CNN takes [B,3,64,64]
        out = model(x)
        energies.append(out['energy'].cpu())      # [B]
        y_true.append(labels)
energies = torch.cat(energies)                  # [N_test]
y_true   = torch.cat(y_true)                    # [N_test]

# 2) Identify top 30% highest‐energy samples as anomalies
#    70th percentile cutoff → top 30% above this
thr = energies.quantile(0.80)


# 3) Predictions
# 3) Logical not then cast
y_pred_inv = (~(energies > thr)).int() #we want 1 for normal, 0 for anomaly. High energy = anomaly. 

# 4) Metrics
print("Threshold for top 30% anomalies:", thr.item())
print("\nConfusion matrix:")
print(confusion_matrix(y_true, y_pred))

acc = (y_true == y_pred).float().mean() * 100
print(f"\nAccuracy: {acc:.2f}%\n")

print("Classification report:")
print(classification_report(y_true, y_pred, target_names=test_ds.classes))

# 5) ROC‑AUC (still informative even though we fix the cutoff by proportion)
auc = roc_auc_score(y_true, -energies)  # invert since lower energy = more normal
print(f"\nROC‑AUC (energy as score): {auc:.3f}")


Threshold for top 30% anomalies: 123.65238952636719

Confusion matrix:
[[20  0]
 [13 87]]

Accuracy: 89.17%

Classification report:
              precision    recall  f1-score   support

   anomalous       0.61      1.00      0.75        20
      normal       1.00      0.87      0.93       100

    accuracy                           0.89       120
   macro avg       0.80      0.94      0.84       120
weighted avg       0.93      0.89      0.90       120


ROC‑AUC (energy as score): 1.000
